# OpenAI Course - Maersk Edition (Complete)
## Comprehensive Guide to OpenAI's API with Maersk Logistics Examples

This notebook covers:
1. Basic Prompting & API Connection
2. Advanced Prompting Techniques
3. Chain of Thought & Self-Evaluation
4. Chat Applications (Simple & Memory)
5. Function Calling & Structured Outputs
6. OpenAI's New Features (GPT-5, Audio, Image, Reasoning)
7. Tool Use & RAG (Retrieval Augmented Generation)

---

## Setup: API Key Configuration

**Choose ONE of the following methods:**

### Method 1: Hardcoded API Key (Quick Start)
Directly set your API key in the code (not recommended for production)

### Method 2: Environment Variable (Recommended)
Use Google Colab's Secrets feature or a .env file

In [ ]:
# Install required packages
!pip install openai python-dotenv -q

In [ ]:
from openai import OpenAI
import os

# =====================================================
# METHOD 1: HARDCODED API KEY (Uncomment to use)
# =====================================================
# OPENAI_API_KEY = "sk-your-api-key-here"  # Replace with your actual API key
# client = OpenAI(api_key=OPENAI_API_KEY)

# =====================================================
# METHOD 2: ENVIRONMENT VARIABLE (Recommended)
# =====================================================
# For Google Colab: Use Secrets (Key icon on left sidebar)
# Add a secret named: OPENAI_API_KEY
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ API Key loaded from Colab Secrets")
except:
    # Fallback: Try environment variable or hardcoded
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'YOUR_API_KEY_HERE')
    if OPENAI_API_KEY == 'YOUR_API_KEY_HERE':
        print("⚠️ WARNING: Please set your API key above in METHOD 1 or use Colab Secrets")
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ API Key loaded from environment or hardcoded value")

print("\n🎯 OpenAI Client Ready!")

---
# Lesson 0: Basic Connection & Simple Prompting

Learn how to:
- Connect to OpenAI API
- Make basic requests
- Use prompt interpolation

In [ ]:
# Basic API call
response = client.responses.create(
    model="gpt-5-nano",
    input="Tell me Maersk company inception story in 200 words max"
)

print("📖 Maersk Story:")
print(response.output_text)

In [ ]:
# Prompt Interpolation Example
religion = "Hindu"
country = "India"
gender = "girl"

def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

prompt = f"""
Suggest five beautiful {gender} baby names.
The names should be popular in {country} and follow {religion} traditions.
Return only the names as a list.
"""

response = get_completion(prompt)
print("👶 Baby Name Suggestions:")
print(response)

---
# Lesson 1: Advanced Prompting Techniques

**Key Principles:**
1. Write clear and specific instructions
2. Use delimiters
3. Ask for structured output (JSON)
4. Check conditions
5. Few-shot prompting

In [ ]:
# Tactic 1: Use delimiters
text = """
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

response = get_completion(prompt)
print("📝 Summary:")
print(response)

In [ ]:
# Tactic 2: Ask for structured output (JSON)
prompt = """
Generate a list of three made-up spare parts for container ships along \
with their manufacturers and primary use cases.
Provide them in JSON format with the following keys:
part_id, name, manufacturer, use_case.
"""

response = get_completion(prompt)
print("📦 Spare Parts (JSON):")
print(response)

In [ ]:
# Tactic 3: Check conditions
inventory = """
{
  "parts": [
    {"part_id": "SP-ENG-001", "name": "Marine Diesel Filter", "stock": {"Mumbai_Warehouse": 45, "Chennai_Depot": 12}},
    {"part_id": "SP-DEC-002", "name": "Hydraulic Pump", "stock": {"Mumbai_Warehouse": 0, "Chennai_Depot": 8}},
    {"part_id": "SP-NAV-003",  "name": "Radar Antenna",   "stock": {"Mumbai_Warehouse": 0, "Chennai_Depot": 0}}
  ]
}
"""

request_text = """
Ship engineer needs part_id=SP-DEC-002 at Mumbai Port.
Preferred pickup location: Mumbai_Warehouse.
"""

prompt = f"""
You will be provided with:
1) A JSON inventory database for ship spare parts
2) A service request

Task:
- If the requested part is available (stock > 0) at the preferred location, output steps for pickup.
- If not available there but available at another location, output steps for transfer.
- If not available anywhere, output "No stock available".

Return your result in the format:
Stock Status: <Available/Not Available>
Location: <Pickup/Transfer/NA>

--- INVENTORY JSON ---
{inventory}

--- SERVICE REQUEST ---
{request_text}
"""

response = get_completion(prompt)
print("🔍 Inventory Check:")
print(response)

In [ ]:
# Tactic 4: Few-shot prompting
text = "Write email on container delay issue."

prompt = f"""
Your task is to write email in a consistent writing style.
The style is short, message-like emails (imperfect English, direct tone).

<email>:
hi sir, container stuck at customs mumbai, we submit docs last week but clearance still pending,
pls check urgent.

<email>:
sir, customer waiting since 3 days, shipment not yet departed from chennai,
vessel schedule changed, pls arrange fast tracking.

<email>:
```{text}```
"""

response = get_completion(prompt)
print("📧 Generated Email:")
print(response)

---
# Lesson 3: Chain of Thought & Self-Evaluation

Give the model time to think:
- Break down complex tasks step-by-step
- Ask model to verify its own work

In [ ]:
# Chain of Thought: Analyzing customer service call
call_transcript = """
Agent: Thank you for calling Maersk Logistics, this is Priya. How may I help you today?
Customer: I'm honestly very frustrated! My container was supposed to arrive three days ago, and I still don't have any updates!
Agent: I'm sorry to hear that, sir. Could you provide me the container number so I can check the status?
Customer: It's MAEU7894561. I've been calling every day, and no one gives me a clear answer!
Agent: I understand how frustrating that must be. Let me check your shipment details… yes, I can see the container is currently held at Nhava Sheva port awaiting customs clearance.
Customer: Customs clearance? Why wasn't I informed about this? My production line is stopped because of this delay!
Agent: I apologize for the lack of communication. It appears there was a documentation issue that's causing the delay.
Customer: Documentation issue? We submitted everything correctly! Are you people even checking properly or just making excuses?
Agent: I assure you, sir, we take every case seriously. I can see that our customs team is working to resolve this, but it may take 2-3 more business days.
Customer: 2-3 more days?! That's unacceptable! I'm losing thousands of rupees every day. I need this resolved immediately!
Agent: I completely understand your concern. Unfortunately, customs procedures are beyond our direct control, but I can escalate this to our port operations manager.
Customer: Escalate it then! This is ridiculous. I pay premium rates for your services and this is what I get?
Agent: I will escalate your case right away, sir. Would you like me to transfer you to my supervisor immediately?
Customer: Yes, do it now. I want to speak to someone who can actually help me, not just give me excuses.
Agent: Understood, sir. I'll transfer your call to my supervisor immediately. Please hold.
"""

enterprise_prompt = f"""
You are analyzing a customer service call transcript. Follow these steps in order to produce a structured JSON report.

<transcript>
{call_transcript}
</transcript>

## Analysis Steps

**Step 1: Validate Data Sufficiency**
First, determine if the transcript contains enough information for analysis.

**Step 2: Analyze Sentiment & Tone**
Evaluate the emotional aspects of the interaction.

**Step 3: Classify the Interaction**
Identify interaction type, urgency, and whether escalation is needed.

**Step 4: Summarize Key Information**
Capture the core issue and resolution.

## Output Requirements
Return ONLY valid JSON with:
- sentiment (overall, customerEmotion, agentTone)
- classification (interactionType, urgency, severity, escalationSuggested)
- summary (customerIssue, resolution, followUpRequired)
"""

response = get_completion(enterprise_prompt)
print("📞 Call Analysis:")
print(response)

---
# Chat Applications

## Simple Chat (No Memory)

In [ ]:
# Simple chat without memory
print("💬 Simple Chat Demo (type 'exit' to quit)\n")

# Uncomment to run interactive chat
# while True:
#     user_input = input("You: ")
#     if user_input.lower() in ["exit", "quit", "bye"]:
#         print("Chat ended.")
#         break
#     
#     response = client.responses.create(
#         model="gpt-5-nano",
#         input=user_input
#     )
#     
#     print("Assistant:", response.output_text)

# Demo example (non-interactive)
demo_input = "What is Maersk's main business?"
response = client.responses.create(
    model="gpt-5-nano",
    input=demo_input
)
print(f"You: {demo_input}")
print(f"Assistant: {response.output_text}")

## Chat with Memory

In [ ]:
# Chat with conversation history
messages = []  # Holds conversation history

print("💬 Chat with Memory Demo\n")

# Demo conversation
demo_messages = [
    "What ports does Maersk operate in India?",
    "Tell me more about the first port you mentioned.",
    "What's the container capacity there?"
]

for user_input in demo_messages:
    messages.append({"role": "user", "content": user_input})
    
    response = client.responses.create(
        model="gpt-5-nano",
        input=messages
    )
    
    reply = response.output_text
    print(f"You: {user_input}")
    print(f"Assistant: {reply}\n")
    
    messages.append({"role": "assistant", "content": reply})

print("✅ Notice how the assistant remembers context from previous messages!")

---
# Function Calling & Structured Outputs

## Custom Function Calling

In [ ]:
import json

# Define tools schema
tools = [
    {
        "type": "function",
        "name": "get_shipment_status",
        "description": "Get the current status and location of a shipment by container number.",
        "parameters": {
            "type": "object",
            "properties": {
                "container_number": {
                    "type": "string",
                    "description": "The container tracking number (e.g., MAEU1234567)",
                },
            },
            "required": ["container_number"],
        },
    },
]

def get_shipment_status(container_number):
    return f"{container_number}: Your container is currently at Mumbai Port, scheduled to depart on 15th October."

# Create input
input_list = [
    {"role": "user", "content": "Where is my shipment? Container number is MAEU1234567."}
]

# Prompt the model with tools
response = client.responses.create(
    model="gpt-5-nano",
    tools=tools,
    input=input_list,
    instructions="Use Tools if you find it necessary."
)

# Save function call outputs
input_list += response.output

print("🔧 Function Calling Demo")
print("\nStep 1: Model decides to call function")

# Execute function if called
for item in response.output:
    if item.type == "function_call":
        if item.name == "get_shipment_status":
            status = get_shipment_status(json.loads(item.arguments))
            
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({"status": status})
            })
            print(f"Step 2: Function executed: {status}")

# Get final response
response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with shipment status information generated by a tool.",
    tools=tools,
    input=input_list,
)

print(f"\nStep 3: Final response to user:")
print(response.output_text)

## Structured Output with Pydantic

In [ ]:
!pip install pydantic -q

from pydantic import BaseModel

class ContainerBooking(BaseModel):
    origin_port: str
    destination_port: str
    departure_date: str
    container_size: str

response = client.responses.parse(
    model="gpt-5-nano",
    input=[
        {"role": "system", "content": "Extract the container booking information."},
        {"role": "user", "content": "I need to book a 40ft container from Mumbai to Rotterdam, departure on 25th October."},
    ],
    text_format=ContainerBooking,
)

print("📋 Structured Output (Pydantic):")
print(response.output_parsed)

---
# OpenAI's Advanced Features

## GPT-5 Model

In [ ]:
# GPT-5 with Responses API
response = client.responses.create(
    model="gpt-5-nano",
    input="Tell me Maersk company inception story in 200 words max"
)

print("🤖 GPT-5 Response:")
print(response.output_text)

## Reasoning Model (Complex Problem Solving)

In [ ]:
# Reasoning model for complex logistics problem
prompt = """
You are a port operations planner for Jawaharlal Nehru Port (JNPT), Mumbai.
There are 250 containers waiting to be loaded onto a vessel. The current loading crane can handle 30 containers per hour.
A cyclone warning has been issued, with the storm expected to hit in 7 hours and lasting for 12 hours. If the vessel doesn't depart before the cyclone, port operations will be suspended, causing a delay fee of ₹8,000 per container.
The total delay cost would be ₹20,00,000 (250 containers × ₹8,000).

Option: The port can deploy an additional mobile harbor crane at ₹1,50,000 per hour, which can load 25 containers per hour.

Question:
1. Should the port deploy the additional crane or not?
2. Show the step-by-step calculation of costs in both scenarios.
3. Recommend the best financial option and explain clearly why.
"""

response = client.responses.create(
    model="gpt-5-nano",
    reasoning={"effort": "medium"},
    input=[{"role": "user", "content": prompt}]
)

print("🧮 Reasoning Model Analysis:")
print(response.output_text)

## Audio Generation

In [ ]:
# Audio generation
response = client.responses.create(
    model="gpt-5-nano",
    input="Generate audio saying: Your container MAEU1234567 has arrived at Mumbai Port and is ready for pickup.",
    tools=[{"type": "audio_generation"}]
)

# Save audio
audio_data = [
    output.result
    for output in response.output
    if output.type == "audio_generation_call"
]

if audio_data:
    with open("shipment_notification.mp3", "wb") as f:
        f.write(audio_data[0])
    print("🔊 Audio saved as: shipment_notification.mp3")
    
    # Play in Colab
    from IPython.display import Audio
    display(Audio("shipment_notification.mp3"))
else:
    print("⚠️ No audio generated")

## Image Generation

In [ ]:
import base64
from IPython.display import Image as IPImage

response = client.responses.create(
    model="gpt-4o-mini",
    input="Generate an image of a large blue Maersk container ship docked at Mumbai Port during sunset",
    tools=[{"type": "image_generation"}],
)

# Save and display image
image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]

if image_data:
    image_base64 = image_data[0]
    with open("maersk_ship.png", "wb") as f:
        f.write(base64.b64decode(image_base64))
    print("🖼️ Image saved as: maersk_ship.png")
    display(IPImage(filename="maersk_ship.png"))
else:
    print("⚠️ No image generated")

---
# Tool Use: Web Search & Code Interpreter

In [ ]:
# Simple Tool Combination
print("🔧 Tool Combination Demo: Web Search + Code Interpreter")
print("=" * 60)

response = client.responses.create(
    model="gpt-5-nano",
    tools=[
        {"type": "web_search"},
        {"type": "code_interpreter", "container": {"type": "auto"}}
    ],
    input="Find the current population of India and calculate how many people that is per square kilometer. India's area is 3.287 million km². Show your calculation step by step."
)

print("✅ Result:")
print(response.output_text)

---
# RAG (Retrieval Augmented Generation)

## File Search with Vector Store

**Note:** To use RAG, you need to:
1. Upload a PDF file to Colab
2. Update the `PDF_PATH` below
3. Run the code

In [ ]:
from pathlib import Path

# Upload your PDF file first, then update this path
PDF_PATH = Path("/content/your_document.pdf")  # Update this!

# Check if file exists
if not PDF_PATH.exists():
    print("⚠️ PDF file not found!")
    print("Please upload a PDF file and update the PDF_PATH variable.")
    print("\nTo upload: Click the folder icon on the left → Upload file")
else:
    print(f"✅ PDF file found: {PDF_PATH}")

In [ ]:
# RAG Demo (only runs if PDF exists)
if PDF_PATH.exists():
    print("\n🔍 Setting up RAG with Vector Store")
    print("=" * 60)
    
    # Create vector store
    try:
        vector_store = client.beta.vector_stores.create(name="maersk_docs_store")
    except AttributeError:
        vector_store = client.vector_stores.create(name="maersk_docs_store")
    
    vector_store_id = vector_store.id
    print(f"✅ Vector store created: {vector_store_id}")
    
    # Upload and index PDF
    print("⏳ Uploading & indexing file...")
    with PDF_PATH.open("rb") as f:
        try:
            client.beta.vector_stores.file_batches.upload_and_poll(
                vector_store_id=vector_store_id,
                files=[f],
            )
        except AttributeError:
            client.vector_stores.file_batches.upload_and_poll(
                vector_store_id=vector_store_id,
                files=[f],
            )
    print("✅ File indexed successfully!\n")
    
    # Demo query
    demo_question = "What are the key points in this document?"
    
    response = client.responses.create(
        model="gpt-5-nano",
        input=demo_question,
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vector_store_id]
        }],
    )
    
    print(f"Question: {demo_question}")
    print(f"\nAnswer: {response.output_text}")
    
    # Check for citations
    if hasattr(response, "citations") and response.citations:
        print("\n📚 Sources:")
        for c in response.citations:
            print(f"  - {c}")
    else:
        print("\n[No citations available]")
else:
    print("⚠️ Skipping RAG demo - no PDF file found")

---
# Summary & Next Steps

## What You Learned:

1. ✅ **Basic Prompting** - Connect to OpenAI API and make simple requests
2. ✅ **Advanced Prompting** - Use delimiters, JSON output, conditions, few-shot learning
3. ✅ **Chain of Thought** - Break down complex problems step-by-step
4. ✅ **Chat Applications** - Build chatbots with and without memory
5. ✅ **Function Calling** - Integrate custom tools and functions
6. ✅ **Structured Outputs** - Use Pydantic for type-safe responses
7. ✅ **Advanced Features** - Audio, image generation, reasoning models
8. ✅ **Tool Use** - Web search, code interpreter
9. ✅ **RAG** - Build document Q&A with vector stores

## Resources:

- [OpenAI Documentation](https://platform.openai.com/docs)
- [OpenAI Cookbook](https://github.com/openai/openai-cookbook)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)

---

**Happy Learning! 🚢**